# Process Bronze

**Objective:** Process the Bronze Layer using a metadata-driven approach.

**Architecture:**
1. **Read Config:** Query `MovieReleases.main.bronze_config`.
2. **Loop & Merge:** Apply SCD2 logic (end-dating old records) to move data from `landing` to `bronze`.


In [1]:
# 1. SETUP & IMPORTS
import duckdb
import pandas as pd
import os
import sys
from dotenv import load_dotenv

# Load Utils
sys.path.append(os.getcwd())
try:
    from utils.db_utils import f_process_scd2
except ImportError:
    print("Error: Could not import utils")

# Load Env (Smart Load)
vLocalEnvPath = r"C:\Users\garym\Documents\GitHub\MovieReleases\.env"
if os.path.exists(vLocalEnvPath):
    load_dotenv(dotenv_path=vLocalEnvPath)
else:
    load_dotenv()

vMdToken = os.getenv("MOTHERDUCK_TOKEN")
if not vMdToken: raise RuntimeError("MOTHERDUCK_TOKEN missing")

# Connect
print("Connecting to MotherDuck...")
con = duckdb.connect(f"md:?motherduck_token={vMdToken}")

Connecting to MotherDuck...


## 2. Execution: Run Merges
Iterate through the config table and execute the merge utility for each row.

In [2]:
# Fetch active configurations
try:
    dfConfig = con.sql("SELECT * FROM MovieReleases.main.bronze_config").df()
except:
    print("Config table not found. Run ingest first.")
    dfConfig = pd.DataFrame()

if dfConfig.empty:
    print("No configuration found. Skipping.")
else:
    print(f"Found {len(dfConfig)} tables to process.")

for index, row in dfConfig.iterrows():
    print(f"Processing Rule: {row['source_table']} -> {row['target_table']} ({row['merge_pattern']})")
    
    if row['merge_pattern'] == 'SCD2':
        # Pass the row as a dictionary to our utility function
        f_process_scd2(con, row.to_dict())

print("Pipeline Complete.")
con.close()

Found 1 tables to process.
Processing Rule: uk_releases -> uk_releases (SCD2)
--- Processing SCD2: MovieReleases.landing.uk_releases -> MovieReleases.bronze.uk_releases ---
Target MovieReleases.bronze.uk_releases does not exist. Initializing from Source...
Initialization Complete.
Pipeline Complete.
